##**Drive Mount**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**Libraries**

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sbn
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = UserWarning)
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
import math
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.preprocessing.image import ImageDataGenerator
import cv2

##**Load Data**

In [5]:
!unzip "/content/drive/MyDrive/archive.zip"

Streaming output truncated to the last 5000 lines.
  inflating: validation_v2/validation/VALIDATION_5003.jpg  
  inflating: validation_v2/validation/VALIDATION_5004.jpg  
  inflating: validation_v2/validation/VALIDATION_5005.jpg  
  inflating: validation_v2/validation/VALIDATION_5006.jpg  
  inflating: validation_v2/validation/VALIDATION_5007.jpg  
  inflating: validation_v2/validation/VALIDATION_5008.jpg  
  inflating: validation_v2/validation/VALIDATION_5009.jpg  
  inflating: validation_v2/validation/VALIDATION_5010.jpg  
  inflating: validation_v2/validation/VALIDATION_5011.jpg  
  inflating: validation_v2/validation/VALIDATION_5012.jpg  
  inflating: validation_v2/validation/VALIDATION_5013.jpg  
  inflating: validation_v2/validation/VALIDATION_5014.jpg  
  inflating: validation_v2/validation/VALIDATION_5015.jpg  
  inflating: validation_v2/validation/VALIDATION_5016.jpg  
  inflating: validation_v2/validation/VALIDATION_5017.jpg  
  inflating: validation_v2/validation/VALIDATION_

In [ ]:
train_dir = os.path.join(os.getcwd(), 'train_v2')
validation_dir = os.path.join(os.getcwd(), 'validation_v2')

In [ ]:
ls

drive/        train_v2/                 written_name_train_v2.csv
sample_data/  validation_v2/            written_name_validation_v2.csv
test_v2/      written_name_test_v2.csv


In [ ]:
train = os.path.join(train_dir, 'train')
valid = os.path.join(validation_dir, 'validation')

In [ ]:
trainY = pd.read_csv('/content/written_name_train_v2.csv')
validY = pd.read_csv('/content/written_name_validation_v2.csv')

## Data cleaning and preprocessing

In [ ]:
trainY.head()

,FILENAME,IDENTITY
0,TRAIN_00001.jpg,BALTHAZAR
1,TRAIN_00002.jpg,SIMON
2,TRAIN_00003.jpg,BENES
3,TRAIN_00004.jpg,LA LOVE
4,TRAIN_00005.jpg,DAPHNE


In [ ]:
validY.head()

,FILENAME,IDENTITY
0,VALIDATION_0001.jpg,BILEL
1,VALIDATION_0002.jpg,LAUMIONIER
2,VALIDATION_0003.jpg,LEA
3,VALIDATION_0004.jpg,JEAN-ROCH
4,VALIDATION_0005.jpg,RUPP


In [ ]:
total_train  = len(os.listdir(train))

total_val = len(os.listdir(valid))

In [ ]:
print(total_train)
print(total_val)

330961
41370


In [ ]:
print("NaNs in train data:",trainY['IDENTITY'].isnull().sum())
print("NaNs in validation data: ", validY['IDENTITY'].isnull().sum())

NaNs in train data: 565
NaNs in validation data:  78


In [ ]:
trainY.dropna(axis=0, inplace=True)
validY.dropna(axis=0, inplace=True)

In [ ]:
trainY = trainY[trainY['IDENTITY'] != 'UNREADABLE']
validY = validY[validY['IDENTITY'] != 'UNREADABLE']

In [ ]:
trainY['Length']=trainY['IDENTITY'].apply(lambda x : len(str(x)))
trainY=trainY[trainY['Length']<=21]
trainY['IDENTITY']=trainY['IDENTITY'].str.upper()
trainY[trainY['Length']==max(trainY['Length'])]

,FILENAME,IDENTITY,Length
10278,TRAIN_10279.jpg,DOSSANTOS LASCONCELOS,21
108567,TRAIN_108568.jpg,O ZUARRDI EREBEVITRAC,21
156500,TRAIN_156501.jpg,ANDRIANNARISOA--DEGHI,21
160575,TRAIN_160576.jpg,LOUISIANE - EVANGELIN,21
174121,TRAIN_174122.jpg,GU ILLOSSOO - OARRIBA,21
212583,TRAIN_212584.jpg,DE MALEZIEUX DU HAMEL,21
220135,TRAIN_220136.jpg,COSTREL DE CORAINVILL,21
229529,TRAIN_229530.jpg,ROUSSEAV -- CHAUDIERE,21
230011,TRAIN_230012.jpg,BEKNARD DE LAVERNETTE,21
308769,TRAIN_308770.jpg,DATE NAISSANCE CLASSE,21


In [ ]:
validY['Length']=validY['IDENTITY'].apply(lambda x : len(str(x)))
validY=validY[validY['Length']<=30]
validY['IDENTITY']=validY['IDENTITY'].str.upper()
validY[validY['Length']==max(validY['Length'])]

,FILENAME,IDENTITY,Length
22648,VALIDATION_22649.jpg,SEMADO TAVARES ANDREA,21


In [ ]:
trainY.reset_index(inplace = True, drop=True) 
validY.reset_index(inplace = True, drop=True) 

In [ ]:
def preprocess(img):
    (h, w) = img.shape    
    final_img = np.ones([64, 64])*255
    if w > 64:
        img = img[:, :64]
        
    if h > 64:
        img = img[:64, :]    
    final_img[:h, :w] = img
    return cv2.rotate(final_img, cv2.ROTATE_90_CLOCKWISE)

In [ ]:
trainY.head()

,FILENAME,IDENTITY,Length
0,TRAIN_00001.jpg,BALTHAZAR,9
1,TRAIN_00002.jpg,SIMON,5
2,TRAIN_00003.jpg,BENES,5
3,TRAIN_00004.jpg,LA LOVE,7
4,TRAIN_00005.jpg,DAPHNE,6


In [ ]:
validY.head()

,FILENAME,IDENTITY,Length
0,VALIDATION_0001.jpg,BILEL,5
1,VALIDATION_0002.jpg,LAUMIONIER,10
2,VALIDATION_0003.jpg,LEA,3
3,VALIDATION_0004.jpg,JEAN-ROCH,9
4,VALIDATION_0005.jpg,RUPP,4


In [ ]:
total_train = 100000
total_valid = len(validY)

In [ ]:
len(trainY)

330285

In [ ]:
train+'/'+trainY.loc[1, 'FILENAME']

'/content/train_v2/train/TRAIN_00002.jpg'

In [ ]:
trainX = []

for i in range(total_train):
    img_dir = train+'/'+trainY.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255.
    trainX.append(image)

In [ ]:
validX = []

for i in range(total_valid):
    img_dir = valid+'/'+validY.loc[i, 'FILENAME']
    image = cv2.imread(img_dir, cv2.IMREAD_GRAYSCALE)
    image = preprocess(image)
    image = image/255.
    validX.append(image)

In [ ]:
trainX = np.array(trainX).reshape(-1, 64, 64, 1)
validX = np.array(validX).reshape(-1, 64, 64, 1)

In [ ]:
trainX[0]

array([[[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       ...,

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]],

       [[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]]])

## CTC Model

In [ ]:
alphabets = "ABCDEFGHIJKLMNOPQRSTUVWXYZ-' "
max_str_len = 24
num_of_characters = len(alphabets) + 1
num_of_timestamps = 64 


def label_to_num(label):
    label_num = []
    for ch in label:
        label_num.append(alphabets.find(ch))
        
    return np.array(label_num)

def num_to_label(num):
    res = ""
    for ch in num:
        if ch == -1: 
            break
        else:
            ret+=alphabets[ch]
    return res

In [ ]:
train_y = np.ones([total_train, max_str_len]) * -1
train_label_len = np.zeros([total_train, 1])
train_input_len = np.ones([total_train, 1]) * (num_of_timestamps-2)
#train_output = np.zeros([total_train])

for i in range(total_train):
    train_label_len[i] = len(trainY.loc[i, 'IDENTITY'])
    train_y[i, 0:len(trainY.loc[i, 'IDENTITY'])]= label_to_num(trainY.loc[i, 'IDENTITY'])  

In [ ]:
valid_y = np.ones([total_valid, max_str_len]) * -1
valid_label_len = np.zeros([total_valid, 1])
valid_input_len = np.ones([total_valid, 1]) * (num_of_timestamps-2)
#valid_output = np.zeros([total_valid])

for i in range(total_valid):
    valid_label_len[i] = len(validY.loc[i, 'IDENTITY'])
    valid_y[i, 0:len(validY.loc[i, 'IDENTITY'])]= label_to_num(validY.loc[i, 'IDENTITY'])   

In [ ]:
print('True label : ',trainY.loc[100, 'IDENTITY'])
print('train_y : ',train_y[100])
print('train_label_len : ',train_label_len[100])
print('\ntrain_input_len : ', train_input_len[100])

True label :  NOUR
train_y :  [13. 14. 20. 17. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1.]
train_label_len :  [4.]

train_input_len :  [62.]


## Basic Model

In [ ]:
classifier = Sequential()

In [ ]:
classifier.add(Conv2D(256, (3, 3), input_shape = (256, 64, 3), activation = 'relu'))
classifier.add(MaxPooling2D(2, 2))

In [ ]:
classifier.add(Conv2D(64, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(2,2))

In [ ]:
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dense(units = 27, activation = 'softmax'))

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 254, 62, 256)      7168      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 127, 31, 256)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 125, 29, 64)       147520    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 62, 14, 64)        0         
_________________________________________________________________
dense (Dense)                (None, 62, 14, 512)       33280     
_________________________________________________________________
dense_1 (Dense)              (None, 62, 14, 27)        13851     
Total params: 201,819
Trainable params: 201,819
Non-trainable params: 0
__________________________________________________